<div style="background-color: steelblue; padding: 10px; border-radius: 5px;">
    <p style="margin: 10;"></p>
    <h1 style="text-align: center; margin: 0; font-weight: bold; color: white;">SWOT Ocean Model Intercomparison Project : EORCA12-ERA5v1 Test Case</h1>
    <p style="margin: 10;"></p>
</div>



## 📦**Imports** 

In [1]:
%%time

##################################
#Imports

from datetime import datetime
import s3fs
import xarray as xr
import pyinterp
from widetrax import DataPreprocessing as dp
from widetrax import Spectra as sp
import numpy as np
import json
from watermark import watermark
import time
import platform

CPU times: user 2.68 s, sys: 1.97 s, total: 4.64 s
Wall time: 2.08 s


## 🔍 **Check the S3 Endpoint**  

In [2]:
s3_folder ="s3://npd-eorca12-era5v1-swot-omip"
endpoint_url =  "https://noc-msm-o.s3-ext.jc.rl.ac.uk"
fs = s3fs.S3FileSystem(anon=True,endpoint_url=endpoint_url)
# List contents of the bucket
bucket_name = 's3://npd-eorca12-era5v1-swot-omip'
contents = fs.ls(bucket_name)
print("Bucket contents:")
for item in contents:
    print(item)

Bucket contents:
npd-eorca12-era5v1-swot-omip/cycle_008
npd-eorca12-era5v1-swot-omip/cycle_009
npd-eorca12-era5v1-swot-omip/cycle_010
npd-eorca12-era5v1-swot-omip/cycle_011
npd-eorca12-era5v1-swot-omip/cycle_012
npd-eorca12-era5v1-swot-omip/cycle_013
npd-eorca12-era5v1-swot-omip/cycle_014
npd-eorca12-era5v1-swot-omip/cycle_015
npd-eorca12-era5v1-swot-omip/cycle_016
npd-eorca12-era5v1-swot-omip/cycle_017
npd-eorca12-era5v1-swot-omip/cycle_018
npd-eorca12-era5v1-swot-omip/cycle_019
npd-eorca12-era5v1-swot-omip/cycle_020
npd-eorca12-era5v1-swot-omip/cycle_021
npd-eorca12-era5v1-swot-omip/cycle_022
npd-eorca12-era5v1-swot-omip/cycle_023
npd-eorca12-era5v1-swot-omip/cycle_024
npd-eorca12-era5v1-swot-omip/cycle_025
npd-eorca12-era5v1-swot-omip/cycle_026


## 🧮 **Required variables**

In [3]:
# REGION A

name_region="Region_B"
season="JFM"

if name_region=="Region_A":
    lon_min = 360-70,
    lon_max = 360-60,
    lat_min = 32,
    lat_max = 42,
elif name_region=="Region_B":
    lon_min = 360-50,
    lon_max = 360-40,
    lat_min = 17,
    lat_max = 27,

area = [lon_min, lat_min, lon_max, lat_max]

if season=="JFM":    
    start_date = "01012024" # "DDMMYYYY"
    end_date ="31032024"
elif season=="JAS":
    start_date = "01072024" # "DDMMYYYY"
    end_date ="30092024"

In [4]:
end_date

'31032024'

## 🔄 **Identify the cycle numbers within the specified time range** 

In [5]:
if season =="JFM":
    file_path = "https://minio.lab.dive.edito.eu/project-meom-ige/cycles_periods.csv" # works only for winter period
elif season =="JAS":
    file_path = "time_ranges.csv"  # for summer

matching_cycles = dp.get_matching_cycles(file_path, start_date, end_date)

def formater_numeros_concis(liste_numeros):
  return [str(numero).zfill(3) for numero in liste_numeros]
    
matching_cycles = formater_numeros_concis(matching_cycles)
matching_cycles


['008', '009', '010', '011', '012', '013']

## 📊 **Load datasets and compute Wavenumber spectra**

In [6]:
%%time

# Load datasets into a dictionary
datasets_dict = dp.read_swot_ncfiles_S3subfolders(
    s3_folder,
    matching_cycles,
    endpoint_url,
    area)

H_008_5 included.
H_008_5 included.
H_008_5 included.
H_008_5 included.
H_008_5 included.
H_009_0 included.
H_009_0 included.
H_009_0 included.
H_009_0 included.
H_009_0 included.
H_009_1 included.
H_009_1 included.
H_009_2 included.
H_009_2 included.
H_009_2 included.
H_009_2 included.
H_009_2 included.
H_009_3 included.
H_009_3 included.
H_009_3 included.
H_009_3 included.
H_009_5 included.
H_009_5 included.
H_009_5 included.
H_009_5 included.
H_009_5 included.
H_010_0 included.
H_010_0 included.
H_010_0 included.
H_010_0 included.
H_010_0 included.
H_010_1 included.
H_010_1 included.
H_010_2 included.
H_010_2 included.
H_010_2 included.
H_010_2 included.
H_010_2 included.
H_010_3 included.
H_010_3 included.
H_010_3 included.
H_010_3 included.
H_010_5 included.
H_010_5 included.
H_010_5 included.
H_010_5 included.
H_010_5 included.
H_011_0 included.
H_011_0 included.
H_011_0 included.
H_011_0 included.
H_011_0 included.
H_011_1 included.
H_011_1 included.
H_011_2 included.
H_011_2 in

In [7]:
%%time 
has_converged, filled_datasets = dp.fill_nan(datasets_dict, varname = "ssh")

CPU times: user 2.56 s, sys: 5.56 s, total: 8.12 s
Wall time: 3.5 s


In [8]:
%%time
segments_dict = sp.retrieve_segments(filled_datasets,FileType = "NetCDF",namevar="ssh")

/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/widetrax/Spectra.py:43: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  for col in range(dataset.dims['num_pixels']):


CPU times: user 268 ms, sys: 5.26 ms, total: 274 ms
Wall time: 273 ms


In [9]:
%%time
psd_dict, freqs_dict = sp.calculate_psd(segments_dict)
# Calculate PSD Mean
psd_mean, freqs_mean = sp.psd_mean_and_freq(psd_dict,freqs_dict)

CPU times: user 759 ms, sys: 505 ms, total: 1.26 s
Wall time: 4.64 s


In [10]:
psd_mean

array([5.90961506e-02, 3.29254400e-01, 3.21693825e-01, 1.29223366e-01,
       3.88877511e-02, 1.19787797e-02, 5.85140928e-03, 3.40200123e-03,
       2.06481141e-03, 1.15005923e-03, 7.71239701e-04, 5.16809848e-04,
       3.48522418e-04, 2.17749465e-04, 1.48894077e-04, 1.02324829e-04,
       6.73343464e-05, 4.54140790e-05, 3.22805331e-05, 2.18908293e-05,
       1.78903816e-05, 1.31410574e-05, 1.02871393e-05, 9.44720819e-06,
       7.52881748e-06, 4.68597986e-06, 2.60018613e-06, 1.66079757e-06,
       1.19530775e-06, 8.64194405e-07, 5.95381693e-07, 4.13509924e-07,
       3.12449994e-07, 2.33125997e-07, 1.79161957e-07, 1.46075391e-07,
       1.20347590e-07, 1.02672118e-07, 9.42187725e-08, 9.26598024e-08,
       8.89879735e-08, 8.71948461e-08, 8.89528909e-08, 8.89739280e-08,
       8.61665492e-08, 8.57988848e-08, 8.97825557e-08, 9.25724241e-08,
       9.72592526e-08, 9.08730176e-08, 7.59306647e-08, 6.02231253e-08,
       5.07673465e-08, 4.25156047e-08, 4.00489453e-08, 4.37857060e-08,
      

## 💾 **Save Results and Information in JSON File**

In [11]:
# Infos
nom_region = name_region
model = "NOC-eORCA12"
name_season = season
freqs_mean = np.array(freqs_mean)
psd_mean = np.array(psd_mean)
todays_Date = datetime.now()
date_creation = todays_Date.isoformat() #ISO 8601 format
packages_versions = watermark(packages="numpy,widetrax,xarray,s3fs,pyinterp,datetime,json,platform,time")
machine_info = {
    "system": platform.system(),
    "node_name": platform.node(),
    "release": platform.release(),
    "version": platform.version(),
    "machine": platform.machine(),
    "processor": platform.processor(),
    "python_version": platform.python_version()
}

# JSON file structure
donnees = {
    "region": nom_region,
    "model": model,
    "lon_min": lon_min,
    "lon_max": lon_max,
    "lat_min": lat_min,
    "lat_max": lat_max,
    "start_date": datetime.strptime(start_date, "%d%m%Y").strftime("%Y-%m-%d"),
    "end_date": datetime.strptime(end_date, "%d%m%Y").strftime("%Y-%m-%d"),
    "packages_versions": packages_versions,
    "date_creation": date_creation,
    "login_creator": platform.node(),
    "Frequency": freqs_mean.tolist(),
    "Spectra": psd_mean.tolist(),
    "machine_info": machine_info
}

# Save on JSON file
with open(str(nom_region)+"_"+str(name_season)+"_"+str(model)+".json", "w", encoding="utf-8") as f:
    json.dump(donnees, f, ensure_ascii=False, indent=4)
#with open("OMIP_"+str(model)+"_"+str(nom_region)+"_"+str(name_season)+".json", "w", encoding="utf-8") as f:
  #  json.dump(donnees, f, ensure_ascii=False, indent=4)

#<region>-<period>-<data-source>.json
print("JSON file created")

JSON file created


## 📤 **Export Results to the S3 Endpoint** 

In [12]:
fs = s3fs.S3FileSystem( anon=True, endpoint_url="https://minio.lab.dive.edito.eu", use_ssl=False ) 

In [13]:
json_file = str(nom_region)+"_"+str(name_season)+"_"+str(model)+".json"
fs.put(json_file, "project-meom-ige/OMIP/")

[None]

In [16]:
fs